In [60]:
from pyspark.sql import SparkSession
from pyspark.mllib.recommendation import ALS    
import pandas as pd

spark = SparkSession.builder.appName('ALS').getOrCreate()

df = pd.read_csv("./ratings.csv", sep=",")

spark_df = spark.createDataFrame(df)

spark_df.show()


+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
| 276733|2080674722|          0|
| 276736|3257224281|          8|
| 276737|0600570967|          6|
| 276744|038550120X|          7|
| 276745| 342310538|         10|
| 276746|0425115801|          0|
| 276746|0449006522|          0|
| 276746|0553561618|          0|
| 276746|055356451X|          0|
| 276746|0786013990|          0|
| 276746|0786014512|          0|
| 276747|0060517794|          9|
| 276747|0451192001|          0|
| 276747|0609801279|          0|
| 276747|0671537458|          9|
+-------+----------+-----------+
only showing top 20 rows



24/01/17 22:46:58 WARN TaskSetManager: Stage 161 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.


In [61]:
from pyspark.sql.functions import lit, col, row_number, desc
from pyspark.sql.window import Window

w = Window().partitionBy()

user_list = spark_df.groupBy("User-ID").count()
book_list = spark_df.groupby("ISBN").count()

user_list = user_list.orderBy(col("count").desc())
book_list = book_list.orderBy(col("count").desc())

user_list = user_list.withColumn("user_index", row_number().over(w.orderBy(desc("count"))))
book_list = book_list.withColumn("book_index", row_number().over(w.orderBy(desc("count"))))


In [62]:
spark_df = spark_df.join(user_list, spark_df["User-ID"] == user_list["User-ID"], "inner").drop(user_list["User-ID"], user_list["count"])
spark_df = spark_df.join(book_list, spark_df["ISBN"] == book_list["ISBN"], "inner").drop(book_list["ISBN"], book_list["count"])
spark_df_als = spark_df.drop('User-ID', 'ISBN')
spark_df.show()

24/01/17 22:47:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:16 WARN TaskSetManager: Stage 162 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:47:17 WARN TaskSetManager: Stage 163 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:47:17 WARN WindowExec: No Partition Define

+-----------+-------+----------+-------------+----------+
|Book-Rating|User-ID|user_index|         ISBN|book_index|
+-----------+-------+----------+-------------+----------+
|          0| 100120|     18279|   )959326839|    150452|
|          9|  71726|     36860|0 440 20615 4|    149370|
|          0| 266641|      3757|0 7525 1962 x|    162695|
|         10|  20919|     14369|0.380.44099.7|    144168|
|          1| 159014|     25014| 000000046565|    153790|
|          8|  11776|     15810| 000001246971|     58172|
|          0| 213324|     12840| 000001246971|     58172|
|          8| 268423|     33351| 000001246971|     58172|
|          5| 242060|     33020|  0000225820X|    160962|
|          0|  33759|      8461|  00006715885|     85284|
|          0| 158856|     40460|  00006715885|     85284|
|          0|  23902|        52|   0001048473|    145904|
|          0|  16319|      4885|   0001372564|    143783|
|          0|  26583|       655|   0001382381|    145735|
|          0| 

In [63]:
spark_df_als.show()

24/01/17 22:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:47:41 WARN TaskSetManager: Stage 196 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:47:42 WARN TaskSetManager: Stage 197 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:47:42 WARN TaskSetManager: Stage 198 conta

+-----------+----------+----------+
|Book-Rating|user_index|book_index|
+-----------+----------+----------+
|          0|     18279|    150452|
|          9|     36860|    149370|
|          0|      3757|    162695|
|         10|     14369|    144168|
|          1|     25014|    153790|
|          8|     15810|     58172|
|          0|     12840|     58172|
|          8|     33351|     58172|
|          5|     33020|    160962|
|          0|      8461|     85284|
|          0|     40460|     85284|
|          0|        52|    145904|
|          0|      4885|    143783|
|          0|       655|    145735|
|          0|       432|    155843|
|          0|      1206|    155984|
|          0|      1009|    151333|
|          0|         1|     58022|
|          0|        88|     58022|
|          0|     40246|     58022|
+-----------+----------+----------+
only showing top 20 rows



In [64]:
model =  ALS.trainImplicit(spark_df_als, rank=5, lambda_=0.01, alpha=1.0, iterations=5)

24/01/17 22:48:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:48:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:48:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:48:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/17 22:48:02 WARN TaskSetManager: Stage 230 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:48:02 WARN TaskSetManager: Stage 231 contains a task of very large size (1875 KiB). The maximum recommended task size is 1000 KiB.
24/01/17 22:48:02 WARN WindowExec: No Partition Define